# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import random as rn
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords','averaged_perceptron_tagger'])

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#%pip install bs4
from bs4 import BeautifulSoup

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report

import string

import warnings
warnings.warn("once")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alby_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alby_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alby_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alby_\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
C:\Users\alby_\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: once


In [2]:
seed = 42
np.random.seed(seed)
rn.seed(seed)

In [3]:
# load data from database
engine = create_engine('sqlite:///ETLmessages.db')
df = pd.read_sql_table('ETLmessages', engine)
X = df['message']
Y = df[df.columns[4:]]

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [20]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [21]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [84]:
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    text = text.translate(str.maketrans('','', string.punctuation))
    
    # tokenize text
    tokens = word_tokenize(text)
    
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [23]:
# test tokenize
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 

Information about the National Palace-
['information', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'sacred', 'heart', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank you!
['please', 'need', 'tent', 'water', 'we', 'silo', 'thank'] 

I would like to receive the messages, thank you
['i', 'would', 'like', 'receive', 'mess

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [30]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),            
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=seed)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=seed)

In [31]:
pipeline.fit(X_train, y_train)

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [32]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
from sklearn.metrics import accuracy_score, classification_report

In [34]:
Y.columns.values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

In [35]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))

The accuracy score is: 0.25328043942630457
Classification Report
                        precision    recall  f1-score   support

               related       0.84      0.91      0.87      4991
               request       0.80      0.45      0.58      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.73      0.60      0.66      2670
          medical_help       0.58      0.12      0.20       535
      medical_products       0.68      0.09      0.16       344
     search_and_rescue       0.48      0.08      0.14       159
              security       0.33      0.01      0.02       116
              military       0.61      0.10      0.16       200
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.41      0.55       418
                  food       0.84      0.46      0.59       745
               shelter       0.80      0.24      0.37       581
              clothing       0.93     

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

In [45]:
parameters = {
    'features__text_pipeline__vect__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__estimator__n_estimators': [200, 500, 1000],
    'clf__estimator__max_depth': [5, 10, 20],
    'clf__estimator__class_weight': ['balanced']
} 

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, cv=5, return_train_score=True, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 49.1min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 336.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
  

In [20]:
y_gscv_pred = cv.predict(X_test)

In [21]:
cv.best_score_

0.2646221137219001

In [22]:
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("{}: {}".format(param_name, best_parameters[param_name])) 

clf__estimator__class_weight: balanced
clf__estimator__max_depth: 20
clf__estimator__n_estimators: 1000
features__text_pipeline__vect__ngram_range: (1, 3)


In [23]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_gscv_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_gscv_pred, target_names=Y.columns.values))

The accuracy score is: 0.2563320109856576
Classification Report
                        precision    recall  f1-score   support

               related       0.92      0.73      0.81      4991
               request       0.66      0.68      0.67      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.77      0.61      0.68      2670
          medical_help       0.53      0.33      0.41       535
      medical_products       0.60      0.34      0.43       344
     search_and_rescue       0.78      0.11      0.20       159
              security       1.00      0.01      0.02       116
              military       0.59      0.26      0.36       200
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.51      0.60       418
                  food       0.71      0.59      0.64       745
               shelter       0.64      0.48      0.55       581
              clothing       0.46      

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [54]:
pipeline_bestparam = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,3))),
            ('tfidf', TfidfTransformer()),            
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight = 'balanced', 
                                                         n_estimators=1000, 
                                                         max_depth=20, 
                                                         n_jobs=-1,random_state=seed)))
])

In [55]:
pipeline_bestparam.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [56]:
y_best_pred = pipeline_bestparam.predict(X_test)

In [60]:
overall_accuracy = (y_best_pred == y_test).mean().mean()
overall_accuracy

0.9418972298511511

In [57]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_best_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_best_pred, target_names=Y.columns.values))

The accuracy score is: 0.2563320109856576
Classification Report
                        precision    recall  f1-score   support

               related       0.92      0.73      0.81      4991
               request       0.66      0.68      0.67      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.77      0.61      0.68      2670
          medical_help       0.53      0.33      0.41       535
      medical_products       0.60      0.34      0.43       344
     search_and_rescue       0.78      0.11      0.20       159
              security       1.00      0.01      0.02       116
              military       0.59      0.26      0.36       200
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.51      0.60       418
                  food       0.71      0.59      0.64       745
               shelter       0.64      0.48      0.55       581
              clothing       0.46      

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Naive Bayes

In [30]:
from sklearn import naive_bayes

In [43]:
naive = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),            
        ]))
    ])),
    ('clf', MultiOutputClassifier(naive_bayes.MultinomialNB()))
])

In [33]:
naive.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [34]:
y_naive_pred = naive.predict(X_test)

In [35]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_naive_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_naive_pred, target_names=Y.columns.values))

The accuracy score is: 0.16112297833384193
Classification Report
                        precision    recall  f1-score   support

               related       0.77      0.99      0.87      4991
               request       0.87      0.20      0.32      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.73      0.64      0.68      2670
          medical_help       0.50      0.00      0.00       535
      medical_products       0.00      0.00      0.00       344
     search_and_rescue       0.00      0.00      0.00       159
              security       0.00      0.00      0.00       116
              military       0.00      0.00      0.00       200
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       418
                  food       0.73      0.01      0.02       745
               shelter       0.00      0.00      0.00       581
              clothing       0.00     

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [36]:
naive.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

In [53]:
parameters_naive = {
    'clf__estimator__alpha': [0.1,0.25,0.5,0.75,1.0],
    'features__text_pipeline__vect__ngram_range': [(1,2),(1,3)]
} 

cv_naive = GridSearchCV(naive, param_grid=parameters_naive, n_jobs=-1, cv=5, return_train_score=True, verbose=2)

In [54]:
cv_naive.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 38.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 63.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
  

In [55]:
y_naivecv_pred = cv_naive.predict(X_test)

In [56]:
cv_naive.best_score_

0.2025226324890652

In [57]:
best_parameters = cv_naive.best_estimator_.get_params()
for param_name in sorted(parameters_naive.keys()):
    print("{}: {}".format(param_name, best_parameters[param_name])) 

clf__estimator__alpha: 0.1
features__text_pipeline__vect__ngram_range: (1, 2)


In [58]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_naivecv_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_naivecv_pred, target_names=Y.columns.values))

The accuracy score is: 0.19484284406469332
Classification Report
                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      4991
               request       0.77      0.54      0.64      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.67      0.71      0.69      2670
          medical_help       0.48      0.09      0.15       535
      medical_products       0.56      0.07      0.12       344
     search_and_rescue       0.40      0.01      0.02       159
              security       0.33      0.01      0.02       116
              military       0.57      0.04      0.07       200
           child_alone       0.00      0.00      0.00         0
                 water       0.55      0.10      0.16       418
                  food       0.73      0.29      0.41       745
               shelter       0.63      0.14      0.23       581
              clothing       0.55     

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### AdaBoost

In [99]:
from sklearn.ensemble import AdaBoostClassifier

In [100]:
pipeline_ada = Pipeline([
    ('features', FeatureUnion([
        
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),            
        ]))
    ])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=seed)))
])

In [101]:
pipeline_ada.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [102]:
y_ada_pred = pipeline_ada.predict(X_test)

In [105]:
overall_accuracy = (y_ada_pred == y_test).mean().mean()
overall_accuracy

0.9471950971416949

In [103]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_ada_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_ada_pred, target_names=Y.columns.values))

The accuracy score is: 0.24992371071101618
Classification Report
                        precision    recall  f1-score   support

               related       0.80      0.96      0.87      4991
               request       0.79      0.50      0.61      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.61      0.67      2670
          medical_help       0.56      0.24      0.33       535
      medical_products       0.66      0.33      0.44       344
     search_and_rescue       0.51      0.15      0.23       159
              security       0.17      0.03      0.06       116
              military       0.56      0.32      0.41       200
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.62      0.67       418
                  food       0.81      0.72      0.76       745
               shelter       0.76      0.52      0.62       581
              clothing       0.58     

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [38]:
pipeline_ada.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fea

In [45]:
parameters_ada = {
    'clf__estimator__learning_rate': [0.01, 0.1],
    'features__text_pipeline__vect__ngram_range': [(1,1),(1,2)]
} 

cv_ada = GridSearchCV(pipeline_ada, param_grid=parameters_ada, n_jobs=-1, cv=5, return_train_score=True, verbose=5)

In [46]:
cv_ada.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed: 22.3min remaining: 22.3min
[Parallel(n_jobs=-1)]: Done  15 out of  20 | elapsed: 26.5min remaining:  8.8min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 35.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 35.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
  

In [47]:
y_adacv_pred = cv_ada.predict(X_test)

In [48]:
cv_ada.best_score_

0.209948123283491

In [51]:
best_parameters = cv_ada.best_estimator_.get_params()
for param_name in sorted(parameters_ada.keys()):
    print("{}: {}".format(param_name, best_parameters[param_name])) 

clf__estimator__learning_rate: 0.1
features__text_pipeline__vect__ngram_range: (1, 1)


In [61]:
overall_accuracy = (y_adacv_pred == y_test).mean().mean()
overall_accuracy

0.9411767198996372

In [52]:
print('The accuracy score is: {}'.format(accuracy_score(y_test.values, y_adacv_pred)))
print('Classification Report')
print(classification_report(y_test.values, y_adacv_pred, target_names=Y.columns.values))

The accuracy score is: 0.21208422337503816
Classification Report
                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4991
               request       0.86      0.26      0.40      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.75      0.41      0.53      2670
          medical_help       0.68      0.04      0.08       535
      medical_products       0.79      0.03      0.06       344
     search_and_rescue       0.89      0.05      0.10       159
              security       0.00      0.00      0.00       116
              military       0.00      0.00      0.00       200
           child_alone       0.00      0.00      0.00         0
                 water       0.81      0.51      0.62       418
                  food       0.80      0.70      0.75       745
               shelter       0.84      0.34      0.48       581
              clothing       0.80     

C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\alby_\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [106]:
import pickle
pickle.dump(pipeline_ada, open('classifier.pkl', "wb" ))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.